# Problemas de estado mutable compartido
En esta sección presentaremos dos problemas, en Java, cada problema contiene el problema del **estado mutable compartido**. La idea es solucionar cada estado mutable compartido utilizando la técnica de copia vista en el vídeo anterior.

## Problema 1. (Sistema de registro de versiones)

Vamos a presentar un problema que es llevar una lista de las diferentes versiones que tenemos almacenadas en un posible fichero.
Empecemos por las clase `Version` la idea de esta clase es tener un registro del número de versión actual (por facilidad con los números de versiones que consta de al menos de tres números en secuencias diferentes X.Y.Z, la vamos a simplicar con único número). Como se puede observar nuestra clase `Version` es una clase muy simple que simplemente tiene dos constructores: Uno por omisión que inicia la secuencia en 0 y otro que inicia con un número de versión que iniciar (Se espera que el número de versión sea un número mayor e igual a 0).

In [14]:
public class Version {
  private int versionNumber;
  public Version(int versionNumber) {
     assert versionNumber >= 0;
     this.versionNumber = versionNumber;
  }
  public Version() {
     this(0);
  }
  public int getVersionNumber() {
     return versionNumber;
  }
  public int incrVersionNumber() {
      return versionNumber++;
  }
  public String toString() {
      return "" + versionNumber;
  }
}

Podemos probar en el siguiente código que va incrementando el número de versión varias veces e imprimiéndolo.

In [13]:
Version v = new Version();
System.out.println(v);
v.incrVersionNumber();
System.out.println(v);
v.incrVersionNumber();
System.out.println(v);

0
1
2


Ahora con este código vamos a implementar una bitácora de versiones sobre un fichero. Normalmente la bitácora de versiones lleva un registro de cada versión en la que ha habido cambios en un proyecto. Para ellos vamos a utilizar una lista (`List`) de versiones donde se guarda cada una de las versiones por las que ha pasado nuestro fichero. Nuestro proyecto consta de un solo fichero. El siguiente código genera la bitácora:

In [17]:
List<Version> log = new ArrayList<>();
Version fileVersion = new Version();

Vamos a simular el comportamiento de la bitácora, realizando una secuencia de 5 cambios a través del siguiente procedimiento que simula la ejecución de n versiones.

In [18]:
static void seqChanges(int n) {
    for (int i = 0; i < n; i++) {
        System.out.println("Current version: " + fileVersion.getVersionNumber());
        fileVersion.incrVersionNumber();
        System.out.println("New version: " + fileVersion.getVersionNumber());
        log.add(fileVersion);
    }
}